In [118]:
#!pip install -U python-dotenv
#!pip install spotipy

In [1]:
import pandas as pd
import os
import plotly.express as px
import spotipy
from spotipy import oauth2
from dotenv import load_dotenv
import os
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
path = os.getcwd()
load_dotenv(dotenv_path=path + f"\env_vars.env")
os.environ['SPOTIPY_CLIENT_ID'] = os.environ.get('SPOTIPY_CLIENT_ID')
os.environ['SPOTIPY_CLIENT_SECRET'] = os.environ.get('SPOTIPY_CLIENT_SECRET')

### Part 1: Loading in the Data
#### We will load in my personal data blah blah, you can replace the string with your file path to analyze your music

In [4]:
num_histories = 4
listening = pd.DataFrame()
for i in range(num_histories):
    temp_df = pd.read_json(path + f"\MyData\StreamingHistory{i}.json")
    listening = pd.concat([listening, temp_df])
print(listening.shape)
listening.head(5)

(33284, 4)


,endTime,artistName,trackName,msPlayed
0,2020-12-26 00:02,Michael Schulte,The Love You Left Behind,194546
1,2020-12-26 00:05,Lewis Capaldi,Lost On You,197178
2,2020-12-26 00:08,Arlissa,Praying for Love,182535
3,2020-12-26 00:12,JP Saxe,A Little Bit Yours,225680
4,2020-12-26 00:16,My Brothers And I,Just Getting to Know You,252840


In [5]:
playlists = pd.read_json(path + f"\MyData\Playlist1.json")
playlists.head(5)

,playlists
0,"{'name': 'My Playlist #23', 'lastModifiedDate'..."
1,"{'name': 'My Playlist #22', 'lastModifiedDate'..."
2,"{'name': 'Stealing Spotify Daily Mixes', 'last..."
3,"{'name': 'Jess Chill Reccomended by Spotify', ..."
4,"{'name': 'Jess <3 Vibes?', 'lastModifiedDate':..."


### Part 2: EDA

In [6]:
playlists['playlists'][3]

{'name': 'Jess Chill Reccomended by Spotify',
 'lastModifiedDate': '2021-12-04',
 'items': [{'track': {'trackName': 'House',
    'artistName': 'Joshua Moss',
    'albumName': 'House',
    'trackUri': 'spotify:track:3BdtiKSZveePCDhfncBOFd'},
   'episode': None,
   'localTrack': None},
  {'track': {'trackName': 'Apple Pie',
    'artistName': 'Lizzy McAlpine',
    'albumName': 'Give Me A Minute',
    'trackUri': 'spotify:track:5FKNq0lLJnP3iZzBTD0PZj'},
   'episode': None,
   'localTrack': None},
  {'track': {'trackName': 'Free',
    'artistName': 'Elina',
    'albumName': 'Free',
    'trackUri': 'spotify:track:3hOWsTM9OpI6c72iQHP4EB'},
   'episode': None,
   'localTrack': None},
  {'track': {'trackName': 'Pancakes for Dinner',
    'artistName': 'Lizzy McAlpine',
    'albumName': 'Give Me A Minute',
    'trackUri': 'spotify:track:4HffeEF97c6UxNePgbuECW'},
   'episode': None,
   'localTrack': None}],
 'description': None,
 'numberOfFollowers': 0}

In [8]:
counts = listening.query('msPlayed >= 3000').groupby(by=['trackName', 'artistName']).size().reset_index()
counts = counts.rename(columns={0: 'count'}).sample(n=100, random_state=42)#.query('count >= 5').sort_values('count', ascending=False) # either listened to >5 times or top x songs or random x songs
counts#.to_csv('grouped.csv')

,trackName,artistName,count
4589,Trying to Kill the Moon,Motherfolk,5
1538,God Knows,Cody Johns,3
315,Avalanche,Handsome and Gretyl,1
297,Asthma Attack,NOAHFINNCE,2
4944,Willow,Jasmine Thompson,4
...,...,...,...
3670,Save My Heart,Jason Reeves,13
3361,Perfect / Can’t Help Falling in Love,Joey Stamper,1
2101,I Would Die 4 U,Noah Guthrie,12
248,Animal (10th Anniversary Edition) - Acoustic,Neon Trees,1


In [11]:
# sp = spotipy.Spotify(auth='TOKEN')
a = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=a)
a = sp.search(q='artist:' + 'Plain White T\'s' + ' track:' + 'Hey There Delilah', type='track')
a['tracks']['items'][0]['uri']
def find_id(song):
    # print(song)
    try:
        song_dict = sp.search(q='artist:' + song[1] + ' track:' + song[0], type='track')
        song_uri = song_dict['tracks']['items'][0]['uri']
    except:
        song_uri = 'DNE'
    return song_uri
counts_with_id = counts[['trackName', 'artistName']].apply(find_id, axis=1)
counts_with_id

4589    (a, spotify:track:5UNC7SQ2z5pnspx4UwyF0O)
1538    (a, spotify:track:6FVfcowRmpcW4BJWjKdF5F)
315     (a, spotify:track:0Kyxsaxjph9rfEru2uXIGu)
297     (a, spotify:track:32V5fBS9jewKqQKj2UYdiI)
4944    (a, spotify:track:0DR1MJq6bTWF8pQa0JmgKS)
                          ...                    
3670    (a, spotify:track:315Tq9xJDQHgxWwSxFbi5e)
3361    (a, spotify:track:0G82D1vFi3D7Zv1KQWA1Kz)
2101    (a, spotify:track:1OXpBGZIPlJfoj7lpn3Vhx)
248     (a, spotify:track:0ZXSZYdrNLu2hciAxcp3ml)
1101    (a, spotify:track:16bjzffqGoXKFQ5DYJlsxU)
Length: 100, dtype: object

In [10]:
counts['trackid'] = counts_with_id
counts.to_csv('tracks_with_id.csv')


### Part 3: Analyzing the Data
#### PCA/plot/etc, figuring out what matters most in a song, what makes a song similar to another?

### Part 3: Loading in a playlist and comparing it to other songs in the playlist
#### Read in, PCA, plot with colors

### Part 4: Selecting additional relevant songs for playlist
#### Find most similar ones to playlist songs